In [39]:
import sys 
import os 
import pandas as pd
import plotly.express as px
import streamlit as st

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from utils.getLatestFiles import get_latest_master_players, get_latest_master_matches, get_latest_master_events


In [40]:
master_players_df = get_latest_master_players()
master_matches_df = get_latest_master_matches()
master_events_df = get_latest_master_events()



✅ 2697 players found in latest MASTER: /home/mdannegarn/table_tennis_stats/Data/Master/Players/20251119_master_players.csv
✅ 23882 matches found in latest MASTER: /home/mdannegarn/table_tennis_stats/Data/Master/Matches/20251120_master_matches.csv 
✅ 188 events found in latest MASTER: /home/mdannegarn/table_tennis_stats/Data/Master/Events/20251119_master_events.csv 


In [41]:
tables_matches_df = pd.merge(master_matches_df, master_events_df[["TableSponsor", "eventId"]], how='left', on='eventId')